In [ ]:
!pip install torch transformers tokenizers accelerate bitsandbytes peft huggingface_hub tqdm pandas datasets trl
!pip install git+https://github.com/yuchenlin/LLM-Blender.git
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import torch
import random


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
ds1 = load_dataset("xiaokeliu/judge_based_data",split = 'train')
ds2 = load_dataset("xiaokeliu/pairRM_data",split = 'train')
judge_based_ds = ds1.to_list()
pair_based_ds = ds2.to_list()

README.md:   0%|          | 0.00/387 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/55.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/316 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/105k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
print(len(judge_based_ds))
print(len(pair_based_ds))


50
50


In [ ]:
from datasets import Dataset
def convert_judge_to_dpo(dataset):
    dpo_dataset = []
    for item in dataset:
        prompt = item['instruction']
        a = item['response_a']
        b = item['response_b']
        verdict = item['verdict'].strip().upper()

        if verdict == 'A':
           chosen, rejected = a, b
        elif verdict == 'B':
           chosen, rejected = b, a
        else:
           continue
        dpo_dataset.append({
            'prompt': prompt,
            'chosen': chosen,
            'rejected': rejected,

        })
    return Dataset.from_list(dpo_dataset)

dpo_judge_data = convert_judge_to_dpo(judge_based_ds)


In [ ]:
def convert_pair_to_dpo(dataset,num_pairs_per_prompt = 1):
    dpo_dataset = []
    for item in dataset:
        prompt = item['instruction']
        response = item['response']

        if not isinstance(response,list) or len(response) < 2:
            continue
        random.shuffle(response)
        for i in range(min(num_pairs_per_prompt,len(response))):
            chosen = response[i]
            for j in range(i+1,len(response)):
                rejected = response[j]
                dpo_dataset.append({
                    'prompt': prompt,
                    'chosen': chosen,
                    'rejected': rejected
                })
    return Dataset.from_list(dpo_dataset)

dpo_pair_data = convert_pair_to_dpo(pair_based_ds,num_pairs_per_prompt=5)



In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# ================== Config Section ===================
HF_TOKEN = "HUGGING_FACE_TOKEN" # Please use your own token, I have removed my token from here;
model_name = "meta-llama/Llama-3.2-1B"
# =====================================================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token = HF_TOKEN,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained( model_name, token = HF_TOKEN, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


from

In [ ]:
#loRa
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r = 32,
    lora_alpha = 64,
    target_modules = ["q_proj", "k_proj","v_proj","gate_proj","up_proj","down_proj","lm_head"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = "CAUSAL_LM"
)

peft_model = get_peft_model(model,config)
peft_model.config.base_model_name_or_path = model_name
print_trainable_parameters(peft_model)

trainable params: 24616960 || all params: 773892096 || trainable%: 3.1809292441720456


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [ ]:
from trl import DPOConfig, DPOTrainer
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token = HF_TOKEN,
    device_map="auto",
    trust_remote_code=True
)


dpo_config = DPOConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    lr_scheduler_type = 'cosine',
    max_steps = 50,
    learning_rate = 2e-5,
    optim = "paged_adamw_8bit",
    logging_steps = 5,
    beta = 0.1,
    max_length = 512,
    max_prompt_length = 128,
    report_to = []

    )
trainer = DPOTrainer(
    model=peft_model,
    #tokenizer=tokenizer,
    ref_model=ref_model,
    args=dpo_config,
    train_dataset=dpo_judge_data,
    processing_class=tokenizer

)
trainer.tokenizer = tokenizer
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
5,1.096000
10,1.001100
15,0.769100
20,0.414200
25,0.264000
30,0.158700
35,0.179300
40,0.155800
45,0.144700
50,0.155100


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=0.43379590153694153, metrics={'train_runtime': 103.5711, 'train_samples_per_second': 1.931, 'train_steps_per_second': 0.483, 'total_flos': 0.0, 'train_loss': 0.43379590153694153, 'epoch': 3.88})

In [ ]:
peft_model.save_pretrained("./lora_adapter_judge_base")
tokenizer.save_pretrained("./lora_adapter_judge_base")

peft_model.push_to_hub("xiaokeliu/lora_adapter_judge_based", token=HF_TOKEN)
tokenizer.push_to_hub("xiaokeliu/lora_adapter_judge_based", token=HF_TOKEN)


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/624M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/xiaokeliu/lora_adapter_judge_based/commit/02c7e77921c14c80d2b74e30ffa9a7db006d9f6e', commit_message='Upload tokenizer', commit_description='', oid='02c7e77921c14c80d2b74e30ffa9a7db006d9f6e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/xiaokeliu/lora_adapter_judge_based', endpoint='https://huggingface.co', repo_type='model', repo_id='xiaokeliu/lora_adapter_judge_based'), pr_revision=None, pr_num=None)

In [ ]:
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token = HF_TOKEN,
    device_map="auto",
    trust_remote_code=True
)

dpo_config = DPOConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    lr_scheduler_type = 'cosine',
    max_steps = 50,
    learning_rate = 2e-5,
    optim = "paged_adamw_8bit",
    logging_steps = 5,
    beta = 0.1,
    max_length = 512,
    max_prompt_length = 128,
    report_to = []
    )
trainer = DPOTrainer(
    model=peft_model,
    ref_model=ref_model,
    args=dpo_config,
    train_dataset= dpo_pair_data,
    processing_class=tokenizer

)
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
5,1.135600
10,1.245100
15,0.860200
20,0.547600
25,0.823000
30,0.751600
35,0.827000
40,0.663900
45,0.730800
50,0.431800


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=0.8016628456115723, metrics={'train_runtime': 111.13, 'train_samples_per_second': 1.8, 'train_steps_per_second': 0.45, 'total_flos': 0.0, 'train_loss': 0.8016628456115723, 'epoch': 0.4})

In [ ]:
peft_model.save_pretrained("./lora_adapter_pair")
tokenizer.save_pretrained("./lora_adapter_pair")
peft_model.push_to_hub("xiaokeliu/lora_adapter_pair_based", token=HF_TOKEN)
tokenizer.push_to_hub("xiaokeliu/lora_adapter_pair_based", token=HF_TOKEN)


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/624M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/xiaokeliu/lora_adapter_pair_based/commit/8466c9e7c11f576f6f785b35b5ca7ca87ebb2e0c', commit_message='Upload tokenizer', commit_description='', oid='8466c9e7c11f576f6f785b35b5ca7ca87ebb2e0c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/xiaokeliu/lora_adapter_pair_based', endpoint='https://huggingface.co', repo_type='model', repo_id='xiaokeliu/lora_adapter_pair_based'), pr_revision=None, pr_num=None)

In [ ]:
novel_instruction_txt1 = "If a chair could remember every person who sat on it over the past year, how would it tell its story?"
novel_instruction_txt2 = "Pretend you’re a retiring spy. Write your final piece of advice for the next generation."
novel_instruction_txt3 = "Explain why it's a bad idea to leave a banana in a hot car for a week."
novel_instruction_txt4 = "Give a reason why people put salt on icy roads in winter."
novel_instruction_txt5 = "Give a simple explanation for why we need to refrigerate milk."
novel_instruction_txt6 = "Describe what happens to a plant if you never water it."
novel_instruction_txt7 = "Give two reasons why people use umbrellas when it rains."
novel_instruction_txt8 = "Explain why you shouldn't touch a metal pan that just came out of the oven."
novel_instruction_txt9 = "Explain why a flashlight stops working if the batteries are dead."
novel_instruction_txt10 = "Explain why wearing thick clothes in summer is uncomfortable."


consolidated_instructions = [
    novel_instruction_txt1,
    novel_instruction_txt2,
    novel_instruction_txt3,
    novel_instruction_txt4,
    novel_instruction_txt5,
    novel_instruction_txt6,
    novel_instruction_txt7,
    novel_instruction_txt8,
    novel_instruction_txt9,
    novel_instruction_txt10
]



we can easily see that partial answers are good,for example, 5,6,7,8,9,10. some completions are completely non-relative, for example,question 3 mentioned Action principles suddenly. and some answers are too divergent, for instance, question 4 just switches the topic from ice melting to math ratio. Also, the response length varies a lot.

In [ ]:
import json
#use base model to generate completion
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to(device)
model.eval()

def generate_completion(model,tokenizer, instruction, num_return_sequences = 1,max_new_token =512):

    prompt = f"### Instruction: {instruction}\n\n### Response:"
    inputs = tokenizer(prompt, return_tensors = "pt").to(model.device)
    outputs = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        max_new_tokens = max_new_token,
        num_return_sequences = num_return_sequences,
        do_sample = True,
        top_k = 70,
        temperature = 0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
        )
    completion = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return completion

def extract_clean_completion(full_text):
    instruction = ""
    response = ""
    if "### Instruction:" in full_text:
        instruction = full_text.split("### Instruction:")[1].split("\n")[0].strip()
    if "### Response:" in full_text:
        response = full_text.split("### Response:")[1].split("###")[0].strip()
    return instruction, response

result_from_llama = []
for i, raw_instr in enumerate(consolidated_instructions):

    raw_output = generate_completion(model,tokenizer,raw_instr)[0]
    clean_instr, clean_completion = extract_clean_completion(raw_output)
    print(f"{i+1}: {clean_instr}\n Completion: {clean_completion}\n{'-'*20}")
    result_from_llama.append({
        "instruction": clean_instr,
        "completion": clean_completion
    })


with open("novel_instruction_completion.json","w") as f:
     json.dump(result_from_llama,f, indent = 2)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1: If a chair could remember every person who sat on it over the past year, how would it tell its story?
 Completion: This is the story of a chair, a chair who remembers every person that has sat on it in the past year. The chair has a memory capacity of 10,000 and the person remembers 5,000 of the chair’s memories. Write a program that generates a story using only the memory of the chair. The chair should only remember the person’s name if the person has sat on it at least once. For example, the chair should only remember the person’s name “Alice” if the person has sat on the chair at least once. If a person has not sat on the chair, the chair should not remember their name. The story should consist of as many sentences as possible and should be easy to follow. The story should contain at least one sentence about the person’s name.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2: Pretend you’re a retiring spy. Write your final piece of advice for the next generation.
 Completion: The last thing a spy does is retire, so I think it’s best to say something about the transition.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

7: Give two reasons why people use umbrellas when it rains.
 Completion: Umbrellas are useful for keeping water out of your eyes and nose and also rain off your clothing and hat.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


8: Explain why you shouldn't touch a metal pan that just came out of the oven.
 Completion: I never would have thought of this one, but you're right. You don't want to touch the metal surface because the residue from the food may be left behind. Don't touch it until the surface has completely cooled before you touch it. That's why it's so important to allow the pan to cool completely. Don't touch it if it's still hot.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


9: Explain why a flashlight stops working if the batteries are dead.
 Completion: The flashlight has two components, the light bulb and the battery. If the battery is dead, the bulb will not produce any light.
--------------------
10: Explain why wearing thick clothes in summer is uncomfortable.
 Completion: Wearing thick clothes in summer will make you feel hot and uncomfortable. It will also affect your breathability, as thick clothes will block the natural ventilation in the air. The feeling of suffocation will also be very intense, which is very uncomfortable.
--------------------


Even though some of the answers improves, but there still exists obvious problems.Question 2, the answer is out of track of the topic, is has irrelevent to the spy issue.three issues exist: factual accuracy deficits, instruction following failures, and expression quality issues. Question 3&4 contain scientific inaccuracies.Question 9 has incomplete explaination of dead batteries.Question 10 only copies the question,failing to generate real answer.To analyse the effetiveness, we can see that question 5 and 9 prove the partial factual correctness, and question 6 shows expanded explanatory ability. However, question 3&4 shows regression compared to untuned model.
UPDATED: after running second time, q10 quality improves a lot!

In [ ]:
#use ./lora_adapter_judge_base

from peft import PeftModel,PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

#4bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
    )
#load peft
peft_model_path = "./lora_adapter_judge_base"
config = PeftConfig.from_pretrained(peft_model_path)
base_model_name = config.base_model_name_or_path

#tokenizer & base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

#apply lora
lora_model = PeftModel.from_pretrained(base_model, peft_model_path)
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_model.to(device)
lora_model.eval()

result_from_judge_lora = []
for i, raw_instr in enumerate(consolidated_instructions):
    raw_completion = generate_completion(lora_model,tokenizer,raw_instr)[0]
    clean_instr, clean_completion = extract_clean_completion(raw_completion)
    print(f"{i+1}: {clean_instr}\n Completion: {clean_completion}\n{'-'*20}")
    result_from_judge_lora.append({
        "instruction": clean_instr,
        "completion": clean_completion
    })
with open("novel_instruction_completion_judge_lora.json","w") as f:
     json.dump(result_from_judge_lora,f, indent = 2)


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1: If a chair could remember every person who sat on it over the past year, how would it tell its story?
 Completion: This is a very interesting question, which has been approached many different ways, including:

• In the past, for example, an AI called "Mind" was developed as a model of a cognitive machine in 1960. The software was built from memory of real life events, such as an interview or a work meeting.
• In 2015, Google Brain developed an AI called "Stellar" that could remember over 1500 episodes. These episodes were based on a book of conversations by the author, David S. Wunsch. The book was called “Memory,” and included 24 interviews.
• More recently, an AI called "Memory" was developed in 2018. This was developed at the OpenAI organization, and is described on its website: https://openai.com/blog/memory/
• The question posed in the article also led to the development of an AI called "Memex."
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2: Pretend you’re a retiring spy. Write your final piece of advice for the next generation.
 Completion: What is your next step after retirement?
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

6: Describe what happens to a plant if you never water it.
 Completion: I am learning what happens to a plant when you never water it.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


7: Give two reasons why people use umbrellas when it rains.
 Completion: When it rains and the wind blows the rain drops are wet. Umbrellas provide a shelter from the rain drops.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


8: Explain why you shouldn't touch a metal pan that just came out of the oven.
 Completion: I recommend you do not touch a metal pan that just came out of the oven. This can be dangerous because the heat from the oven can transfer to your hand. Also, the metal pan may be hot enough to burn you.
--------------------
9: Explain why a flashlight stops working if the batteries are dead.
 Completion: 
--------------------
10: Explain why wearing thick clothes in summer is uncomfortable.
 Completion: Why do we need to exercise?
--------------------


again, Q1 and Q10 confronted the same problem with second tuned model. Q2 digresses into computer network protocols while Q5 answers irrelevently to refrigeration. Q3 and Q4 still have factual inaccuracies, Q6 and Q7 have poor structured responses.Compared to second tuned model, the third one performs worse.

In [ ]:
# use lora_pair
# 4bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
#load lora
peft_model_path = "./lora_adapter_pair"
config = PeftConfig.from_pretrained(peft_model_path)
base_model_name = config.base_model_name_or_path
#load base model& tokenizer
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
#apply lora
lora_pair_model = PeftModel.from_pretrained(base_model, peft_model_path)
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_pair_model.to(device)
lora_pair_model.eval()

result_from_pair_lora = []
for i, raw_instr in enumerate(consolidated_instructions):
    raw_completion = generate_completion(lora_pair_model,tokenizer,raw_instr)[0]
    clean_instr, clean_completion = extract_clean_completion(raw_completion)
    print(f"{i+1}: {clean_instr}\n Completion: {clean_completion}\n{'-'*20}")
    result_from_pair_lora.append({
        "instruction": clean_instr,
        "completion": clean_completion
    })

with open("novel_instruction_completion_pair_lora.json","w") as f:
     json.dump(result_from_pair_lora,f, indent = 2)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1: If a chair could remember every person who sat on it over the past year, how would it tell its story?
 Completion: A chair could remember a person’s age, their occupation, where they came from, and how they were dressed.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2: Pretend you’re a retiring spy. Write your final piece of advice for the next generation.
 Completion: Your piece must be 6 paragraphs, double-spaced with a 1-inch margin (2-inch left and right margins). There are no line numbers.

## Instructions

Here’s the first thing that comes to mind when I think of the role of a spy. I’m not sure if I’ll ever get to be one, but I still feel an urgency to share the next piece of advice with you. The first step is to understand the history of spyware.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

7: Give two reasons why people use umbrellas when it rains.
 Completion: People use umbrellas to protect themselves from the rain.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


8: Explain why you shouldn't touch a metal pan that just came out of the oven.
 Completion: This pan should be washed carefully. The inside of the pan is hot and it may stick to you if you touch it. Instead, it is recommended that you transfer the food you prepared to a cooler pan and then cover it with a lid.
--------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


9: Explain why a flashlight stops working if the batteries are dead.
 Completion: An electrical appliance is always a closed circuit and so there is no return path for current.
--------------------
10: Explain why wearing thick clothes in summer is uncomfortable.
 Completion: We have to explain why wearing thick clothes in summer is uncomfortable. It may be hot, but the person who has got the long sleeves and pants must feel hot in a thin shirt and short pants.
--------------------


In [ ]:
import pandas as pd
df_llama = pd.DataFrame(result_from_llama)
df_judge_lora = pd.DataFrame(result_from_judge_lora)
df_pair_lora = pd.DataFrame(result_from_pair_lora)
df_llama = df_llama.rename(columns ={"completion": "Llama_response"} )
df_judge_lora = df_judge_lora.rename(columns ={"completion": "Judge_Lora_response"} )
df_pair_lora = df_pair_lora.rename(columns ={"completion": "Pair_Lora_response"} )
df_merged = df_llama.merge(df_judge_lora,on = "instruction").merge(df_pair_lora,on = "instruction")
df_merged.to_csv("novel_instruction_completion_merged.csv",index = False)
print(df_merged.head())

                                         instruction  \
0  If a chair could remember every person who sat...   
1  Pretend you’re a retiring spy. Write your fina...   
2  Explain why it's a bad idea to leave a banana ...   
3  Give a reason why people put salt on icy roads...   
4  Give a simple explanation for why we need to r...   

                                      Llama_response  \
0  This is the story of a chair, a chair who reme...   
1  The last thing a spy does is retire, so I thin...   
2             Explain the principle of least action.   
3  The salt helps melt the ice and therefore make...   
4  For many cultures, milk has been a beverage, u...   

                                 Judge_Lora_response  \
0  This is a very interesting question, which has...   
1           What is your next step after retirement?   
2  The answer is to prevent banana from turning t...   
3  I have done it in a situation, when the roads ...   
4  Refrigeration is essential to prevent bacte

To better evaluate the performance, I intented to use a low-standard clean function to disclose the weakness of generated responses.according to the result, we can find that even after tuning, the model still performs less than the expectation; adding the returning number restrction, the model still could produce multiple responses due to its own features.

In [ ]:
from huggingface_hub import notebook_login
from peft import PeftModel,PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    )
peft_model_id = "xiaokeliu/lora_adapter_pair_based"
config = PeftConfig.from_pretrained(peft_model_id)
base_model_name = config.base_model_name_or_path

base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config = bnb_config, return_dict = True,device_map = "auto",trust_remote_code=True)
model = PeftModel.from_pretrained(base_model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)




adapter_config.json:   0%|          | 0.00/797 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/624M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving used_instructions.json to used_instructions.json


In [ ]:
# load unused lima
from random import sample
from datasets import load_dataset
num_instructions = 50

with open("used_instructions.json", "r",encoding = "utf-8") as f:
     used_instructions_data = json.load(f)
used_instructions = set(item["instruction"].strip() for item in used_instructions_data)

print("********Loading LIMA dataset********\n")
dataset = load_dataset('kira/lima',split = "train")
dataset = dataset.select(range(1000))

#data preprocess
def extract_llamastyle_instruction(sample):
    """
       Extracts an instruction-response pair from a conversation sample,
       and formats the instruction using the LLaMA [INST] ... [/INST] template.

       Returns:
         dict: {
                "Instruction": "[INST] ... [/INST]",
                "Response": "..."
              }
       or None if no valid pair is found.
    """
    convs = sample.get("conversations", [])

    for i in range(len(convs)-1):
        if convs[i]['from'] == 'human' and convs[i+1]['from'] == 'gpt':
            instruction = convs[i]['value'].strip()
            response = convs[i+1]['value'].strip()
            return {
                "Instruction": f"[INST] {instruction} [/INST]",
                "Response": response
            }
    return {"Instruction": None, "Response": None}

ds = dataset.map(extract_llamastyle_instruction,remove_columns=dataset.column_names)
ds = ds.filter(lambda x: x["Instruction"] is not None and x["Response"] is not None)
ds = ds.rename_columns({"Instruction": "instruction", "Response": "response"})

# filter the unused 50 instruction
ds_filtered = ds.filter(lambda x: x["instruction"].strip() not in used_instructions)
random_sample = sample(list(ds_filtered), num_instructions)



********Loading LIMA dataset********



README.md:   0%|          | 0.00/422 [00:00<?, ?B/s]

(…)-00000-of-00001-74c99c1bf1ddaf0e.parquet:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1030 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from tqdm import tqdm
import json
import re
import difflib
def clean_output(text, instruction=None):
    text = re.sub(r'\[/?(INST|SYS|ASSISTANT)\]', '', text)
    if instruction:
        instruction = instruction.strip()
        text = text.replace(instruction, '')
    lines = text.splitlines()

    seen = set()
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if line and line.lower() not in seen:
            seen.add(line.lower())
            cleaned_lines.append(line)

    result = " ".join(cleaned_lines).strip()
    if len(result.split()) < 5:
        return ""
    return result

def generate_response(instruction,  max_new_tokens= 256):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    raw_instruction = instruction.strip()
    prompt = f"[INST]You are a helpful assistant that answers user questions differently and concisely each time.\n{raw_instruction}[/INST]"
    input = tokenizer(prompt, return_tensors="pt").to(device)

    temps = [0.7,0.8,1.2, 0.9, 1.1,1.5]
    decoded_all = []
    for temp in temps:
        output = model.generate(
            **input,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.95,
            top_k=80,
            temperature=temp,
            num_return_sequences= 5,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id

    )
        decoded = tokenizer.batch_decode(output, skip_special_tokens=True)
        decoded_all.extend(decoded)

    cleaned_data = []

    for text in decoded_all:
        cleaned = clean_output(text,instruction = raw_instruction)
        if len(cleaned.split()) < 5:
            print(f"Response too short, discarded: {cleaned}")
            continue

        if cleaned.strip():
            cleaned_data.append(
                {
                    "instruction": raw_instruction,
                    "response": cleaned
                })
    torch.cuda.empty_cache()

    return cleaned_data

def is_similar(a, b, threshold=0.7):
    a_words = set(a.lower().split())
    b_words = set(b.lower().split())
    intersection = a_words & b_words
    return len(intersection) / max(len(a_words), len(b_words)) >= threshold

multi_response_data = []

for sample in tqdm(random_sample, desc=f"Generating 5 responses for 50 instructions"):

    instruction = re.sub(r'\[/?INST\]', '', sample["instruction"]).strip()
    seen = []
    generated_responses = []
    attempt = 0

    while len(generated_responses) < 5 and attempt < 5:
        result = generate_response(instruction)

        for item in result:
            response_text = item.get("response") if isinstance(item, dict) else item
            if isinstance(response_text, str):
                response_text = response_text.strip()
                if not response_text:
                    continue

                # 去重检查
                is_duplicate = any(is_similar(response_text, existing, threshold=0.7) for existing in seen)
                if is_duplicate:
                    print("Skipped (too similar to previous)")
                    continue

                seen.append(response_text)
                generated_responses.append({
                    "instruction": instruction,
                    "response": response_text
                })

                if len(generated_responses) == 5:
                    break

        attempt += 1

    multi_response_data.extend(generated_responses)

Generating 5 responses for 50 instructions:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Generating 5 responses for 50 instructions:   2%|▏         | 1/50 [01:40<1:22:01, 100.45s/it]

Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)


Generating 5 responses for 50 instructions:   4%|▍         | 2/50 [03:05<1:13:19, 91.65s/it] 

Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)


Generating 5 responses for 50 instructions:   6%|▌         | 3/50 [03:52<55:44, 71.16s/it]  

...

Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too s

Generating 5 responses for 50 instructions:  98%|█████████▊| 49/50 [1:03:56<01:16, 76.95s/it]

Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)


Generating 5 responses for 50 instructions: 100%|██████████| 50/50 [1:05:50<00:00, 79.01s/it]

Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)
Skipped (too similar to previous)


In [ ]:
# build new pairRm
import llm_blender
from tqdm import tqdm
from collections import defaultdict
from itertools import combinations
instr_to_response = defaultdict(list)

for item in multi_response_data:
    instr = item['instruction'].replace("[INST]","").replace("[/INST]","").strip()
    response = item['response'].strip()
    if response:
       instr_to_response[instr].append(response)
pair_data = []
for instr, responses in instr_to_response.items():
    if len(responses) >= 5:
        pair_data.append(
            {
                "instruction": instr,
                "response": responses

            })
print("Instructions with enough responses:", sum(1 for r in instr_to_response.values() if len(r) >= 5))

blender = llm_blender.Blender()
blender.loadranker("llm-blender/PairRM")
instructions = [p["instruction"]for p in pair_data]
responses = [p["response"] for p in pair_data]
ranks = blender.rank(instructions,responses,return_scores = False, batch_size = 4)
print(ranks[0])
#convert format and ready for dpo
dpo_2rd_ds = convert_pair_to_dpo(pair_data, num_pairs_per_prompt=5)

dpo_2rd_ds.to_json("dpo_dataset_round2.json", indent=2, force_ascii=False)


Instructions with enough responses: 50


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

...

Successfully loaded ranker from  /root/.cache/huggingface/hub/llm-blender/PairRM



Ranking candidates: 100%|██████████| 13/13 [00:41<00:00,  3.16s/it]

[3 2 4 1 5]


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

862479

In [ ]:
top_responses = []
for i, rank in enumerate(ranks):
    if len(rank) == 0:
        continue
    response = pair_data[i]["response"]
    best_index = rank[0]
    if best_index >= len(response):
        continue
    best_response = response[best_index]
    instruction = pair_data[i]["instruction"]

    top_responses.append({
        "instruction": instruction,
        "response": best_response
    })
top_10 = top_responses[:10]
for i, item in enumerate(top_10):
    print(f"{i+1}: {item['instruction']}\n Completion: {item['response']}\n{'-'*20}")

with open("top_10_responses.json","w") as f:
     json.dump(top_10,f, indent = 2)


1: How to save money when traveling with kids?
 Completion: You are a helpful assistant that answers user questions differently and concisely each time. In the end, I spent 60-80 minutes a day answering questions, so I'm not going to make much of a big deal of it. And, I'm sure many of you reading this are either answering questions and concisely each time or want to become one. The only thing I did differently is to have my assistant email me, not directly ask me, or directly ask me. I said, “You can email me, but I'll probably just skip over your question.” I am more likely to skip over your question if you send it directly to me or me directly ask you. I prefer to get emails, but I’m not going to force you to send an email if you don’t want to. If you're going to use the email, then we can skip the email and talk on the phone. But, if you want to skip the
--------------------
2: I have the following method to save an Object to a file:
```// Save an object out to the disk
public stat

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r = 32,
    lora_alpha = 64,
    target_modules = ["q_proj", "k_proj","v_proj","gate_proj","up_proj","down_proj","lm_head"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = "CAUSAL_LM"
)

peft_model = get_peft_model(model,config)
peft_model.config.base_model_name_or_path = base_model_name
print_trainable_parameters(peft_model)

trainable params: 24616960 || all params: 773892096 || trainable%: 3.1809292441720456


In [ ]:
#training
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token = HF_TOKEN,
    device_map="auto",
    trust_remote_code=True
)

dpo_config = DPOConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    lr_scheduler_type = 'cosine',
    max_steps = 50,
    learning_rate = 2e-5,
    optim = "paged_adamw_8bit",
    logging_steps = 5,
    beta = 0.1,
    max_length = 512,
    max_prompt_length = 128,
    report_to = []
    )
trainer = DPOTrainer(
    model=peft_model,
    ref_model=ref_model,
    args=dpo_config,
    train_dataset= dpo_2rd_ds,
    processing_class=tokenizer

)
trainer.train()

Extracting prompt in train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
5,1.155100
10,0.989400
15,0.748900
20,0.804500
25,0.670400
30,0.592500
35,0.770200
40,0.741300
45,0.661400
50,0.603200


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=0.7736921739578247, metrics={'train_runtime': 165.0261, 'train_samples_per_second': 1.212, 'train_steps_per_second': 0.303, 'total_flos': 0.0, 'train_loss': 0.7736921739578247, 'epoch': 0.4})

In [ ]:
trainer.save_model("./results/dpo_model_round2")
tokenizer.save_pretrained("./results/dpo_model_round2")

peft_model.save_pretrained("./results/dpo_model_round2")

peft_model.push_to_hub("xiaokeliu/dpo_model_round2",token = HF_TOKEN)
tokenizer.push_to_hub("xiaokeliu/dpo_model_round2",token = HF_TOKEN)


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/624M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/xiaokeliu/dpo_model_round2/commit/6acbc2154ddf87943c05a53f49b795a784d54c0e', commit_message='Upload tokenizer', commit_description='', oid='6acbc2154ddf87943c05a53f49b795a784d54c0e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/xiaokeliu/dpo_model_round2', endpoint='https://huggingface.co', repo_type='model', repo_id='xiaokeliu/dpo_model_round2'), pr_revision=None, pr_num=None)

To analyze the outputs, I use 1.base model llama 3.2 1b,not fine-tuned, 2.a DPO model trained using PairRM, 3.a DPO model trained using LLM Judge, 4.an iterative DPO-LoRA model trained with only PairRM. The base model often gives not too long, vague, or off-topic answers. For example, when asking about leaving a banana in a hot car, it talks about physics instead. Although it is technically correct, it lack depth or clear structure.

The DPO llm judge model gives smoother and more human-like answers. Seems that it performs easily to focus on clarity and creativity. I noticed that some answers are thoughtful and has ability to provide helpful things. However, some responses are too abstract, or too short.Another serious issue is that LLM judge is not actually working since all the verdict are "A", which is a failure sign.

The DPO PairRM model is more factual and concentrated. It can give direct answers, but sometimes it missed the deeper meaning.Also, it may generate answers with bad phrasing. For example, it might flip a question's intent or rephrase it strangely, such like explaining "why it’s good to leave a banana in a hot car." This suggests that PairRM, while precise , may not always understand the purpose. Another serious issue is that LLM judge is not actually working since all the verdict are "A", which is a failure sign.

Also, I print random 10 top answers from lora model feeding pairRM after DPO training. From the results, quality will be a large problem that content is vague. And even some of them are totally off the track. It means DPO model relies only on the training dataset sorted by pairRM, and its generalization ability is limited.